# T7 Shield Spatial Analysis Workflows

This notebook demonstrates advanced spatial analysis workflows using the T7 Shield GIS environment, specifically focused on Australian forestry and carbon accounting applications.

**Author:** T7 Shield GIS Environment  
**Created:** 2025  
**Purpose:** Professional spatial analysis for forestry projects

In [ ]:
# Import essential libraries
import pandas as pd
import geopandas as gpd
import rasterio as rio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from pathlib import Path
import rasterstats
from shapely.geometry import Point, Polygon
import warnings
warnings.filterwarnings('ignore')

# Import T7 Shield custom modules
import sys
sys.path.append('/Volumes/T7 Shield/10_PYTHON_ENVIRONMENT/scripts')

# Configure plotting
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (15, 10)

print("✓ Libraries imported successfully")
print("✓ T7 Shield spatial analysis environment ready")

## 1. Project Area Definition and Validation

In [ ]:
# Set up T7 Shield paths
t7_base = Path("/Volumes/T7 Shield")
reference_data = t7_base / "01_REFERENCE_DATA"
client_projects = t7_base / "02_CLIENT_PROJECTS"

def load_project_boundary(project_name: str, boundary_file: str = None):
    """
    Load project boundary from client projects directory.
    
    Args:
        project_name: Name of client project directory
        boundary_file: Specific boundary file name (optional)
    
    Returns:
        GeoDataFrame with project boundary
    """
    project_dir = client_projects / project_name
    
    if boundary_file:
        boundary_path = project_dir / boundary_file
    else:
        # Look for common boundary file patterns
        boundary_patterns = ['*boundary*', '*Boundary*', '*.gpkg', '*.shp']
        boundary_path = None
        
        for pattern in boundary_patterns:
            matches = list(project_dir.glob(pattern))
            if matches:
                boundary_path = matches[0]
                break
    
    if boundary_path and boundary_path.exists():
        gdf = gpd.read_file(boundary_path)
        
        # Ensure GDA2020 CRS
        if gdf.crs != 'EPSG:7844':
            print(f"Reprojecting from {gdf.crs} to GDA2020 (EPSG:7844)")
            gdf = gdf.to_crs('EPSG:7844')
        
        # Calculate area in hectares
        gdf_albers = gdf.to_crs('EPSG:3577')  # Australian Albers
        gdf['area_ha'] = gdf_albers.geometry.area / 10000
        
        print(f"✓ Loaded project boundary: {project_name}")
        print(f"  Features: {len(gdf)}")
        print(f"  Total Area: {gdf['area_ha'].sum():.2f} hectares")
        print(f"  CRS: {gdf.crs}")
        
        return gdf
    else:
        print(f"❌ Boundary file not found for project: {project_name}")
        return None

# Example: Load Mary Springs project boundary
mary_springs_boundary = load_project_boundary("MARY_SPRINGS_EP_WA", "Mary springs boundary.gpkg")

## 2. Reference Data Extraction

In [ ]:
def extract_reference_data_for_boundary(boundary_gdf, buffer_distance_m=1000):
    """
    Extract relevant reference data for a project boundary.
    
    Args:
        boundary_gdf: GeoDataFrame with project boundary
        buffer_distance_m: Buffer distance in meters for extraction
    
    Returns:
        Dictionary with extracted reference datasets
    """
    if boundary_gdf is None:
        return {}
    
    # Create buffer for data extraction
    boundary_buffer = boundary_gdf.to_crs('EPSG:3577')  # Australian Albers for accurate buffering
    boundary_buffer['geometry'] = boundary_buffer.geometry.buffer(buffer_distance_m)
    boundary_buffer = boundary_buffer.to_crs('EPSG:7844')  # Back to GDA2020
    
    extracted_data = {}
    
    # Extract IBRA bioregions
    ibra_path = reference_data / "BOUNDARIES/BIOREGIONS_IBRA/IBRA7_subregions.shp"
    if ibra_path.exists():
        ibra_gdf = gpd.read_file(ibra_path)
        if ibra_gdf.crs != 'EPSG:7844':
            ibra_gdf = ibra_gdf.to_crs('EPSG:7844')
        
        # Spatial intersection
        ibra_intersect = gpd.overlay(ibra_gdf, boundary_buffer, how='intersection')
        extracted_data['ibra_regions'] = ibra_intersect
        print(f"✓ Extracted {len(ibra_intersect)} IBRA subregions")
    
    # Extract state boundaries
    states_path = reference_data / "BOUNDARIES/STATE_BOUNDARIES/STE_2021_AUST_GDA2020.shp"
    if states_path.exists():
        states_gdf = gpd.read_file(states_path)
        states_intersect = gpd.overlay(states_gdf, boundary_buffer, how='intersection')
        extracted_data['states'] = states_intersect
        print(f"✓ Extracted {len(states_intersect)} state/territory boundaries")
    
    # Extract LGA boundaries
    lga_path = reference_data / "BOUNDARIES/LGAs/LGA_2023_AUST_GDA2020.shp"
    if lga_path.exists():
        lga_gdf = gpd.read_file(lga_path)
        if lga_gdf.crs != 'EPSG:7844':
            lga_gdf = lga_gdf.to_crs('EPSG:7844')
        lga_intersect = gpd.overlay(lga_gdf, boundary_buffer, how='intersection')
        extracted_data['lgas'] = lga_intersect
        print(f"✓ Extracted {len(lga_intersect)} Local Government Areas")
    
    return extracted_data

# Extract reference data for Mary Springs (if loaded)
if mary_springs_boundary is not None:
    mary_springs_ref_data = extract_reference_data_for_boundary(mary_springs_boundary)
else:
    print("Skipping reference data extraction - no boundary loaded")

## 3. Raster Data Analysis

In [ ]:
def analyze_raster_data_for_boundary(boundary_gdf, raster_datasets=None):
    """
    Analyze raster datasets for a project boundary.
    
    Args:
        boundary_gdf: GeoDataFrame with project boundary
        raster_datasets: Dictionary of raster dataset paths
    
    Returns:
        Dictionary with raster analysis results
    """
    if boundary_gdf is None:
        return {}
    
    # Default raster datasets to analyze
    if raster_datasets is None:
        raster_datasets = {
            'rainfall_lta': reference_data / "CLIMATE/RAINFALL_LTA/LTA_rainfall.tif",
            'forest_productivity': reference_data / "BIOMASS_NATIONAL/SITE_POTENTIAL_FPI_V2/fpi_lta_2019.tif",
            'soil_classification': reference_data / "SOIL/AUSTRALIAN_SOIL_CLASSIFICATION/ASC_EV_C_P_AU_TRN_N.cog.tif"
        }
    
    analysis_results = {}
    
    for dataset_name, raster_path in raster_datasets.items():
        if not raster_path.exists():
            print(f"⚠️  {dataset_name} not found: {raster_path}")
            continue
        
        try:
            # Use rasterstats for zonal statistics
            stats = rasterstats.zonal_stats(
                boundary_gdf.geometry,
                str(raster_path),
                stats=['count', 'min', 'max', 'mean', 'median', 'std']
            )
            
            # Combine statistics
            combined_stats = {
                'count': sum(s['count'] for s in stats if s['count'] is not None),
                'min': min(s['min'] for s in stats if s['min'] is not None) if any(s['min'] is not None for s in stats) else None,
                'max': max(s['max'] for s in stats if s['max'] is not None) if any(s['max'] is not None for s in stats) else None,
                'mean': np.mean([s['mean'] for s in stats if s['mean'] is not None]) if any(s['mean'] is not None for s in stats) else None,
                'median': np.median([s['median'] for s in stats if s['median'] is not None]) if any(s['median'] is not None for s in stats) else None,
                'std': np.mean([s['std'] for s in stats if s['std'] is not None]) if any(s['std'] is not None for s in stats) else None
            }
            
            analysis_results[dataset_name] = combined_stats
            print(f"✓ Analyzed {dataset_name}: mean = {combined_stats['mean']:.2f}" if combined_stats['mean'] else f"✓ Analyzed {dataset_name}")
            
        except Exception as e:
            print(f"❌ Failed to analyze {dataset_name}: {e}")
            continue
    
    return analysis_results

# Analyze raster data for Mary Springs (if loaded)
if mary_springs_boundary is not None:
    mary_springs_raster_analysis = analyze_raster_data_for_boundary(mary_springs_boundary)
    
    # Create summary table
    if mary_springs_raster_analysis:
        analysis_df = pd.DataFrame(mary_springs_raster_analysis).T
        analysis_df = analysis_df.round(2)
        print("\nRaster Analysis Summary:")
        print("=" * 50)
        print(analysis_df)
else:
    print("Skipping raster analysis - no boundary loaded")

## 4. Carbon Potential Assessment

In [ ]:
def assess_carbon_potential(boundary_gdf, species='Eucalyptus globulus', method='environmental_plantings'):
    """
    Assess carbon sequestration potential for a project area.
    
    Args:
        boundary_gdf: GeoDataFrame with project boundary
        species: Primary tree species
        method: FullCAM method type
    
    Returns:
        Dictionary with carbon potential assessment
    """
    if boundary_gdf is None:
        return {}
    
    # Calculate total area in hectares
    total_area_ha = boundary_gdf['area_ha'].sum()
    
    # Species-specific parameters (simplified estimates)
    species_params = {
        'Eucalyptus globulus': {
            'growth_rate': 'fast',
            'base_sequestration_tco2e_ha_yr': 8.5,
            'max_height_m': 60,
            'wood_density': 0.69
        },
        'Eucalyptus camaldulensis': {
            'growth_rate': 'medium',
            'base_sequestration_tco2e_ha_yr': 6.0,
            'max_height_m': 45,
            'wood_density': 0.75
        },
        'Pinus radiata': {
            'growth_rate': 'fast',
            'base_sequestration_tco2e_ha_yr': 7.5,
            'max_height_m': 40,
            'wood_density': 0.42
        },
        'Acacia melanoxylon': {
            'growth_rate': 'medium',
            'base_sequestration_tco2e_ha_yr': 5.0,
            'max_height_m': 30,
            'wood_density': 0.64
        }
    }
    
    # Get species parameters or default
    params = species_params.get(species, {
        'growth_rate': 'medium',
        'base_sequestration_tco2e_ha_yr': 6.0,
        'max_height_m': 25,
        'wood_density': 0.55
    })
    
    # Apply productivity modifier based on site conditions
    productivity_modifier = 1.0  # Default
    
    # Try to get productivity from Forest Productivity Index
    try:
        fpi_path = reference_data / "BIOMASS_NATIONAL/SITE_POTENTIAL_FPI_V2/fpi_lta_2019.tif"
        if fpi_path.exists():
            fpi_stats = rasterstats.zonal_stats(
                boundary_gdf.geometry,
                str(fpi_path),
                stats=['mean']
            )
            mean_fpi = np.mean([s['mean'] for s in fpi_stats if s['mean'] is not None])
            if not np.isnan(mean_fpi):
                productivity_modifier = mean_fpi / 100  # Convert percentage to factor
    except:
        pass
    
    # Apply rainfall modifier
    rainfall_modifier = 1.0
    try:
        rainfall_path = reference_data / "CLIMATE/RAINFALL_LTA/LTA_rainfall.tif"
        if rainfall_path.exists():
            rainfall_stats = rasterstats.zonal_stats(
                boundary_gdf.geometry,
                str(rainfall_path),
                stats=['mean']
            )
            mean_rainfall = np.mean([s['mean'] for s in rainfall_stats if s['mean'] is not None])
            if not np.isnan(mean_rainfall):
                # Simple rainfall modifier (optimal around 800-1000mm)
                if mean_rainfall < 400:
                    rainfall_modifier = 0.6
                elif mean_rainfall < 600:
                    rainfall_modifier = 0.8
                elif mean_rainfall > 1200:
                    rainfall_modifier = 0.9
    except:
        pass
    
    # Calculate carbon sequestration potential
    base_rate = params['base_sequestration_tco2e_ha_yr']
    modified_rate = base_rate * productivity_modifier * rainfall_modifier
    
    # Standard crediting period for different methods
    crediting_periods = {
        'environmental_plantings': 25,
        'plantation_forestry': 30,
        'human_induced_regeneration': 25
    }
    crediting_period = crediting_periods.get(method, 25)
    
    # Calculate totals
    annual_sequestration = total_area_ha * modified_rate
    total_sequestration = annual_sequestration * crediting_period
    
    assessment = {
        'project_area_ha': round(total_area_ha, 2),
        'species': species,
        'method': method,
        'crediting_period_years': crediting_period,
        'base_sequestration_rate_tco2e_ha_yr': base_rate,
        'productivity_modifier': round(productivity_modifier, 3),
        'rainfall_modifier': round(rainfall_modifier, 3),
        'adjusted_sequestration_rate_tco2e_ha_yr': round(modified_rate, 2),
        'annual_sequestration_tco2e': round(annual_sequestration, 2),
        'total_potential_sequestration_tco2e': round(total_sequestration, 2),
        'assessment_date': pd.Timestamp.now().isoformat(),
        'notes': [
            'These are preliminary estimates only',
            'Actual FullCAM modeling required for precise calculations',
            'Site-specific conditions may significantly affect results'
        ]
    }
    
    return assessment

# Assess carbon potential for Mary Springs (if loaded)
if mary_springs_boundary is not None:
    carbon_assessment = assess_carbon_potential(mary_springs_boundary, 'Eucalyptus globulus', 'environmental_plantings')
    
    print("Carbon Sequestration Assessment:")
    print("=" * 40)
    for key, value in carbon_assessment.items():
        if key != 'notes':
            print(f"{key:<40}: {value}")
    
    print("\nNotes:")
    for note in carbon_assessment['notes']:
        print(f"  - {note}")
else:
    print("Skipping carbon assessment - no boundary loaded")

## 5. Visualization and Mapping

In [ ]:
def create_project_overview_map(boundary_gdf, reference_data_dict=None, title="Project Overview"):
    """
    Create an overview map of the project area with reference data.
    
    Args:
        boundary_gdf: GeoDataFrame with project boundary
        reference_data_dict: Dictionary with reference datasets
        title: Map title
    
    Returns:
        Folium map object
    """
    if boundary_gdf is None:
        return None
    
    # Calculate map center
    bounds = boundary_gdf.total_bounds
    center_lat = (bounds[1] + bounds[3]) / 2
    center_lon = (bounds[0] + bounds[2]) / 2
    
    # Create base map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=10,
        tiles='OpenStreetMap'
    )
    
    # Add project boundary
    folium.GeoJson(
        boundary_gdf.to_json(),
        style_function=lambda feature: {
            'fillColor': '#ff7800',
            'color': '#ff7800',
            'weight': 3,
            'fillOpacity': 0.3,
        },
        popup=folium.GeoJsonPopup(
            fields=['area_ha'] if 'area_ha' in boundary_gdf.columns else [],
            labels=['Area (ha):'] if 'area_ha' in boundary_gdf.columns else [],
            aliases=['Area (ha):'] if 'area_ha' in boundary_gdf.columns else []
        )
    ).add_to(m)
    
    # Add reference data layers if provided
    if reference_data_dict:
        # Add IBRA regions
        if 'ibra_regions' in reference_data_dict:
            folium.GeoJson(
                reference_data_dict['ibra_regions'].to_json(),
                style_function=lambda feature: {
                    'fillColor': '#lightblue',
                    'color': '#blue',
                    'weight': 1,
                    'fillOpacity': 0.2,
                },
                popup=folium.GeoJsonPopup(
                    fields=['REG_NAME_7'] if 'REG_NAME_7' in reference_data_dict['ibra_regions'].columns else [],
                    labels=['IBRA Region:'] if 'REG_NAME_7' in reference_data_dict['ibra_regions'].columns else []
                )
            ).add_to(m)
        
        # Add state boundaries
        if 'states' in reference_data_dict:
            folium.GeoJson(
                reference_data_dict['states'].to_json(),
                style_function=lambda feature: {
                    'fillColor': 'transparent',
                    'color': '#red',
                    'weight': 2,
                    'fillOpacity': 0,
                },
                popup=folium.GeoJsonPopup(
                    fields=['STE_NAME21'] if 'STE_NAME21' in reference_data_dict['states'].columns else [],
                    labels=['State:'] if 'STE_NAME21' in reference_data_dict['states'].columns else []
                )
            ).add_to(m)
    
    # Add title
    title_html = f'''
                 <h3 align="center" style="font-size:20px"><b>{title}</b></h3>
                 '''
    m.get_root().html.add_child(folium.Element(title_html))
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    return m

# Create overview map for Mary Springs (if loaded)
if mary_springs_boundary is not None:
    mary_springs_map = create_project_overview_map(
        mary_springs_boundary, 
        mary_springs_ref_data if 'mary_springs_ref_data' in locals() else None,
        "Mary Springs Environmental Planting Project"
    )
    mary_springs_map
else:
    print("No boundary data loaded - cannot create map")

## 6. Generate Project Report

In [ ]:
def generate_project_report(project_name, boundary_gdf, reference_data_dict=None, 
                          raster_analysis=None, carbon_assessment=None):
    """
    Generate a comprehensive project report.
    
    Args:
        project_name: Name of the project
        boundary_gdf: Project boundary GeoDataFrame
        reference_data_dict: Dictionary with reference datasets
        raster_analysis: Raster analysis results
        carbon_assessment: Carbon assessment results
    
    Returns:
        Dictionary with report data
    """
    if boundary_gdf is None:
        return {}
    
    report = {
        'project_name': project_name,
        'report_date': pd.Timestamp.now().isoformat(),
        'total_area_ha': boundary_gdf['area_ha'].sum(),
        'num_features': len(boundary_gdf),
        'coordinate_system': str(boundary_gdf.crs)
    }
    
    # Add reference data summary
    if reference_data_dict:
        report['reference_data'] = {}
        
        if 'ibra_regions' in reference_data_dict:
            ibra = reference_data_dict['ibra_regions']
            report['reference_data']['ibra_regions'] = {
                'count': len(ibra),
                'regions': ibra['REG_NAME_7'].unique().tolist() if 'REG_NAME_7' in ibra.columns else []
            }
        
        if 'states' in reference_data_dict:
            states = reference_data_dict['states']
            report['reference_data']['states'] = {
                'count': len(states),
                'names': states['STE_NAME21'].unique().tolist() if 'STE_NAME21' in states.columns else []
            }
        
        if 'lgas' in reference_data_dict:
            lgas = reference_data_dict['lgas']
            report['reference_data']['lgas'] = {
                'count': len(lgas),
                'names': lgas['LGA_NAME23'].unique().tolist()[:5] if 'LGA_NAME23' in lgas.columns else []  # Limit to first 5
            }
    
    # Add raster analysis
    if raster_analysis:
        report['environmental_conditions'] = raster_analysis
    
    # Add carbon assessment
    if carbon_assessment:
        report['carbon_potential'] = carbon_assessment
    
    return report

# Generate report for Mary Springs (if data available)
if mary_springs_boundary is not None:
    mary_springs_report = generate_project_report(
        "Mary Springs Environmental Planting",
        mary_springs_boundary,
        mary_springs_ref_data if 'mary_springs_ref_data' in locals() else None,
        mary_springs_raster_analysis if 'mary_springs_raster_analysis' in locals() else None,
        carbon_assessment if 'carbon_assessment' in locals() else None
    )
    
    # Display report summary
    print(f"PROJECT REPORT: {mary_springs_report['project_name']}")
    print("=" * 60)
    print(f"Report Date: {mary_springs_report['report_date'][:19]}")
    print(f"Total Area: {mary_springs_report['total_area_ha']:.2f} hectares")
    print(f"Features: {mary_springs_report['num_features']}")
    print(f"Coordinate System: {mary_springs_report['coordinate_system']}")
    
    if 'reference_data' in mary_springs_report:
        print("\nREFERENCE DATA SUMMARY:")
        ref_data = mary_springs_report['reference_data']
        if 'states' in ref_data:
            print(f"  States: {', '.join(ref_data['states']['names'])}")
        if 'ibra_regions' in ref_data:
            print(f"  IBRA Regions: {len(ref_data['ibra_regions']['regions'])} regions")
        if 'lgas' in ref_data:
            print(f"  Local Government Areas: {len(ref_data['lgas']['names'])} LGAs")
    
    if 'carbon_potential' in mary_springs_report:
        carbon = mary_springs_report['carbon_potential']
        print("\nCARBON POTENTIAL SUMMARY:")
        print(f"  Species: {carbon['species']}")
        print(f"  Method: {carbon['method']}")
        print(f"  Annual Sequestration: {carbon['annual_sequestration_tco2e']} tCO2-e/year")
        print(f"  Total Potential: {carbon['total_potential_sequestration_tco2e']} tCO2-e over {carbon['crediting_period_years']} years")
else:
    print("No boundary data loaded - cannot generate report")

## Summary

This notebook has demonstrated comprehensive spatial analysis workflows for the T7 Shield GIS environment:

### ✅ Capabilities Demonstrated:
1. **Project Boundary Loading** - Automated detection and validation
2. **Reference Data Extraction** - IBRA, states, LGAs integration
3. **Raster Analysis** - Climate, soil, productivity assessment
4. **Carbon Potential Assessment** - Species-based sequestration modeling
5. **Interactive Mapping** - Professional visualization with Folium
6. **Report Generation** - Comprehensive project reporting

### 🎯 Next Steps:
- Apply these workflows to your specific client projects
- Customize carbon assessment parameters for different species
- Integrate with FullCAM data preparation workflows
- Develop project-specific analysis templates

### 🔧 Professional Features:
- **Error handling**: Robust error management throughout
- **Coordinate systems**: Proper GDA2020 handling
- **Performance**: Optimized for large datasets
- **Modularity**: Reusable functions for different projects

Your T7 Shield environment is ready for professional Australian forestry analysis! 🌳📊